In [11]:
import random
import torch

from utils.Logger import showPlot
from utils.Timer import asMinutes, timeSince

%load_ext autoreload
from data.exampleJP_Data import Lang, prepareData
from model.seq2seq_Model import (
    Seq2Seq_LSTM_Attn_ptModel, 
    Seq2SeqTranslate_ptTokenizer,
)
from baselineJP_ExpTrain import example_ExpTrain
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load data

In [12]:
!mkdir _data_example/
%cd '_data_example'

if True:    # 解凍する場合は True に書き換え
    !git clone https://github.com/odashi/small_parallel_enja
    !echo '------------------'
    !head -n 5 small_parallel_enja/train.en
    !head -n 5 small_parallel_enja/train.ja
    !echo '------------------'
!ls
%cd ../

mkdir: _data_example/: File exists
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment/_data_example
fatal: destination path 'small_parallel_enja' already exists and is not an empty directory.
------------------
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
emi looks happy .
please bear this fact in mind .
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
エミ は 幸せ そう に 見え ま す 。
この 事実 を 心 に 留め て お い て 下さ い 。
------------------
data                data.zip            small_parallel_enja
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment


In [13]:
input_lang, output_lang, pairs = prepareData('en', 'ja', '_data_example/small_parallel_enja', False)
print(random.choice(pairs))

Reading lines...
Read 50000 sentence pairs
Counting words...
Counted words:
en 6637
ja 8777
['he was also kind to animals .', '彼 は また 動物 たち に 優し かっ た 。']


In [14]:
# train / test split
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(pairs, test_size=0.2)

## setup Experiment

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## setup Data

In [16]:
## Data
input_lang, output_lang, pairs = prepareData(
        'en', 'ja', '_data_example/small_parallel_enja', True)
print(random.choice(pairs))

Reading lines...
Read 50000 sentence pairs
Counting words...
Counted words:
ja 8777
en 6637
['胸焼け が し ま す 。', 'i have heartburn .']


## setup Model

In [17]:
## Model
tokenizer = Seq2SeqTranslate_ptTokenizer(
                    input_lang, output_lang, device)
seq2seq_model = Seq2Seq_LSTM_Attn_ptModel(
                    input_lang.n_words, output_lang.n_words, hidden_size,
                    tokenizer, device, dropout_p=0.1)

NameError: name 'hidden_size' is not defined

## setup exec

In [10]:
## exec
exp_train = example_ExpTrain(pairs)
exp_train.exec(seq2seq_model, 50,
                print_every=100, plot_every=10, 
                learning_rate=0.01, log_dir='_logs')
# 本実験
# exp_train.exec(seq2seq_model, 75000,
#                 print_every=5000, plot_every=1000, 
#                 learning_rate=0.01, log_dir='_logs')

NameError: name 'seq2seq_model' is not defined

___

## evaluate
- random print
- Acc
- BLEU などの 評価指標

In [31]:
%load_ext autoreload
from baseline_ExpEval import eval_print_randomly
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
eval_print_randomly(seq2seq_model, pairs)

> je suis sure que tom va le faire .
= i m sure that tom will do that .
< you re . 

> je ne suis pas facilement offensee .
= i m not easily offended .
< you re . 

> elle est en relation avec cette societe .
= she is connected with that company .
< you re . 

> elle a presse un citron pour le the .
= she squeezed a lemon for tea .
< you re . 

> je suis heureux que tu aies mentionne cela .
= i m glad you brought this up .
< you re . 

> je suis surprise de vous voir .
= i m surprised to see you .
< you re . 

> il est reporter pour le time .
= he is a reporter for time magazine .
< you re . 

> tu te fais comprendre .
= you re assertive .
< you re . 

> ils n ont pas froid aux yeux .
= they re brave .
< you re . 

> ils sont tous les memes .
= they are all the same .
< you re . 

